In [1]:
25*6/10

15.0

In [2]:
from gmshtools.CircRect6 import CircRect


In [3]:
import gmsh 
import numpy as np

gmsh.initialize()

gmsh.model.add("t1")

# cilindro
Ly = 25
Lx = 60
tn  = 3.0
tl = 0.5
r  = 0.32

A = np.pi*r**2

factor = 2.2
rx = factor*r 
ry = (1/factor)*r

R = 10/6
nx_fibres = 5


z0 = tn/2
dx = 0.1 
dz = 0.1

hx = R
hz = tl
Lz = 2*2*hz + 2*z0 

def CreateCylinder_Lxx(x0,z0):

    ellipse_1 = CircRect(x0, 0, z0, 
                         rx, ry, 
                         [0,1,0] ,            # vec_z 
                         [1,0,0])             # vec_x
    
    ellipse_2 = CircRect(x0, +Ly/2, z0,
                            rx, ry, 
                            [0,1,0] ,            # vec_z 
                            [1,0,0])             # vec_x
    gmsh.model.occ.synchronize()

    thrusection = gmsh.model.occ.addThruSections([ellipse_1, ellipse_2])

    gmsh.model.occ.synchronize()

    return thrusection[0][1]

def CreateCylinder_Lyy(y0,z0):
    
    ellipse_1 = CircRect(0, y0, z0,
                            rx, ry, 
                            [1,0,0] ,            # vec_z 
                            [0,1,0])             # vec_x
    
    ellipse_2 = CircRect(nx_fibres*hx, y0, z0,
                            rx, ry, 
                            [1,0,0] ,            # vec_z 
                            [0,1,0])             # vec_x
    
    gmsh.model.occ.synchronize()



    thrusection = gmsh.model.occ.addThruSections([ellipse_1, ellipse_2])

    gmsh.model.occ.synchronize()

    return thrusection[0][1]

FP1_list = []
FM1_list = []
for ix in range(nx_fibres):
    FibrePlus_01  = CreateCylinder_Lxx(hx/2 + hx*ix, +z0 + 3*hz/2 )
    FibreMinus_01 = CreateCylinder_Lxx(hx/2 + hx*ix, -z0 - 3*hz/2 )

    FP1_list.append(FibrePlus_01)
    FM1_list.append(FibreMinus_01)



gmsh.model.occ.synchronize()
# 
#

nfilaments = int(0.5*25*6/10)

yspan = np.arange(0, Ly/2 - hx, hx)
yspan = yspan + hx/2

FP2_list = []
FM2_list = []
for iy in yspan:
    FibrePlus_02  = CreateCylinder_Lyy(iy, +z0 + hz/2 )
    FibreMinus_02 = CreateCylinder_Lyy(iy, -z0 - hz/2 )

    FP2_list.append(FibrePlus_02)
    FM2_list.append(FibreMinus_02)

FP2_list.extend(FP1_list)
FM2_list.extend(FM1_list)

# add Physical Groups
fibres_tags = np.concatenate([FP1_list, FM1_list, FP2_list, FM2_list])
f1 = gmsh.model.addPhysicalGroup(3, fibres_tags)
gmsh.model.setPhysicalName(3, f1, "Fibres")


gmsh.model.occ.synchronize()
box_01 = gmsh.model.occ.addBox(0, 0 , + z0 ,
                               nx_fibres*hx   ,  Ly/2   ,  2*hz    )

box_02 = gmsh.model.occ.addBox(0, 0 , - z0 ,
                               nx_fibres*hx   ,  Ly/2   ,  -2*hz    )

gmsh.model.occ.synchronize()

box_01_cut = gmsh.model.occ.cut([(3, box_01)], 
                   [ (3,i) for i in FP2_list],
                   removeTool=False)

box_02_cut = gmsh.model.occ.cut([(3, box_02)], 
                   [(3,i) for i in FM2_list],
                   removeTool=False)




gmsh.model.occ.synchronize()



In [4]:



# # box 
# #
# gmsh.model.occ.synchronize()
# #

box_big_mid = gmsh.model.occ.addBox(0      , 0     , -z0,
                                    Lx/2   ,  Ly/2   ,  2*z0    )


box_sup = gmsh.model.occ.addBox(0       ,   0     , z0 + hz,
                                Lx/2    ,  Ly/2   ,  hz       )

box_inf = gmsh.model.occ.addBox(0       ,   0     , -z0 - hz,
                                Lx/2    ,  Ly/2   ,  -hz       )

gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(3)



box_big = gmsh.model.occ.addBox(0     ,  0    , -Lz/2,
                                Lx/2  ,  Ly/2   ,  Lz    )

gmsh.model.occ.synchronize()

# cut box_big - volumes
box_big = gmsh.model.occ.fragment([(3,box_big)], volumes)



gmsh.model.occ.synchronize()



# buscar la caja que tiene como media z= 0


In [5]:
volumes = gmsh.model.getEntities(3)
com = [ gmsh.model.occ.getCenterOfMass(3, i[1]) 
       for i in volumes ]


sort_com_index = np.argsort(np.abs(np.array(com)[:,2]))

In [6]:
id = volumes[sort_com_index[0]][1]

In [7]:
# physical group
f2 = gmsh.model.addPhysicalGroup(3, [id])
gmsh.model.setPhysicalName(3, f2, "Nucleo")
# 
gmsh.model.occ.synchronize()

#
resina = [ i[1] for i in volumes if i[1] != id ]
resina = [ i for i in resina if i not in fibres_tags]


# sort by x
com = [ gmsh.model.occ.getCenterOfMass(3, i) 
       for i in resina ]

sort_com_index = np.argsort(np.array(com)[:,0])

resina = [resina[i] for i in sort_com_index]


In [8]:
resina

[32, 29, 28, 34, 31, 33, 30, 35]

In [9]:
resina[:4]

[32, 29, 28, 34]

In [10]:

f3 = gmsh.model.addPhysicalGroup(3, resina[:4])
gmsh.model.setPhysicalName(3, f3, "Resina")

capas_hom = resina[4:]
# sort 
com = [ gmsh.model.occ.getCenterOfMass(3, i) for i in capas_hom ]
# sort by z
sort_com_index = np.argsort(np.array(com)[:,2])

capas_hom = [capas_hom[i] for i in sort_com_index]

f4 = gmsh.model.addPhysicalGroup(3, [capas_hom[0],capas_hom[3]])
gmsh.model.setPhysicalName(3, f4, "CAPAS_2")

f5 = gmsh.model.addPhysicalGroup(3, [capas_hom[1],capas_hom[2]])
gmsh.model.setPhysicalName(3, f5, "CAPAS_1")


#

In [11]:
# gmsh.fltk.run()

In [12]:
# set mesh 
# gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.25*r)
# gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 4.0*r)

gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.35*r)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 8.0*r)

gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 10) # 20



    #
field = gmsh.model.mesh.field
field.add("MathEval", 1)

THX = "(0.5 + 0.5*Tanh(20*(-x + x0)) ) "
THZ_PLUS  = "( 0.5 + 0.5*Tanh(20*( z - z0)) )"
THZ_MINUS = "( 0.5 + 0.5*Tanh(20*(-z - z0)) )"
Fz = "0.8 - 0.45 * THX * (THZ_PLUS + THZ_MINUS)"

Fz = Fz.replace("THX", THX)
Fz = Fz.replace("THZ_PLUS", THZ_PLUS)
Fz = Fz.replace("THZ_MINUS", THZ_MINUS)
Fz = Fz.replace("z0", str(z0))
Fz = Fz.replace("x0", str(nx_fibres*hx))
field.setString(1, "F", Fz)

gmsh.fltk.run()

# Aplicar el campo de fondo
field.setAsBackgroundMesh(1)


gmsh.model.mesh.generate(3)

# second order 
gmsh.model.mesh.setOrder(2)
gmsh.write("t1.inp")

In [13]:
E1 = 8000 
E2 = 700 

Ec = 8000

nu = 0.3 
Em = 1000
En = 800


In [14]:
from djccx.inp.inp import  inp 

inp_f = inp("t1.inp")

inp_f.remove_by_type(2)

nodes = inp_f.nodes.df

nodes_sym = nodes.copy()
nodes_sym = nodes_sym[ nodes_sym["x"] < 1e-3] 

nset_sym_x = inp_f.CreateNsetFromIds(nodes_sym.index, "sym_x")

nodes_sym = nodes.copy()
nodes_sym = nodes_sym[ nodes_sym["y"] < 1e-3] 

nset_sym_y = inp_f.CreateNsetFromIds(nodes_sym.index, "sym_y")

nodes_fixed = nodes.copy()
nodes_fixed = nodes_fixed[ nodes_fixed["x"] > 0.5*Lx  - 1e-3]
nodes_fixed = nodes_fixed[ nodes_fixed["z"] < -0.5*Lz + 1e-3]

nset_fixed = inp_f.CreateNsetFromIds(nodes_fixed.index, "fixed")

nodes_force = nodes.copy()
nodes_force = nodes_force[ nodes_force["x"] < 1e-3]
nodes_force = nodes_force[ nodes_force["z"] > 0.5*Lz - 1e-3]

nset_force = inp_f.CreateNsetFromIds(nodes_force.index, "force")

# materials 
mat_fibras = inp_f.CreateElasticMaterial("FIBRES", Ec, nu)
mat_resina = inp_f.CreateElasticMaterial("RESINA", Em, nu)
mat_capas_1 = inp_f.CreateElasticMaterial("CAPAS_1", E1, nu)
mat_capas_2 = inp_f.CreateElasticMaterial("CAPAS_2", E1, nu)
mat_nucleo = inp_f.CreateElasticMaterial("NUCLEO", En, nu)


# elsets 
fibres_elset = inp_f.select("FIBRES","elset")
resina_elset = inp_f.select("RESINA","elset")
capas_1_elset = inp_f.select("CAPAS_1","elset")
capas_2_elset = inp_f.select("CAPAS_2","elset")
nucleo_elset = inp_f.select("NUCLEO","elset")

inp_f.CreateSolidSection( fibres_elset, mat_fibras)
inp_f.CreateSolidSection( resina_elset, mat_resina)
inp_f.CreateSolidSection( capas_1_elset, mat_capas_1)
inp_f.CreateSolidSection( capas_2_elset, mat_capas_2)
inp_f.CreateSolidSection( nucleo_elset, mat_nucleo)
#

istep = inp_f.CreateStaticStep()
istep.CreateBoundary(nset_fixed,3,0)
istep.CreateBoundary(nset_sym_x,1,0)
istep.CreateBoundary(nset_sym_y,2,0)

disp = 5
istep.CreateBoundary(nset_force, 3, -disp)


inp_f.print("t1_new.inp")

In [15]:
fibres_elset = inp_f.select("FIBRES","elset")


In [16]:
inp_f.elsets

[Card (*ELSET) :FIBRES,
 Card (*ELSET) :NUCLEO,
 Card (*ELSET) :RESINA,
 Card (*ELSET) :CAPAS_2,
 Card (*ELSET) :CAPAS_1]

In [17]:
0.5*Lz - 1e-3

2.499

In [18]:
# import gmsh 
# import numpy as np

# gmsh.initialize()
# x0 = 0
# y0 = 0
# z0 = 0
# Lx = 60
# Ly = 25
# x_eps = 1e-4

# vec_x = np.array([1, 0, 0])
# vec_z = np.array([0, 0, 1])

# vec_z = vec_z / np.linalg.norm(vec_z)
# vec_x = vec_x / np.linalg.norm(vec_x)


# vec_y = np.cross(vec_z, vec_x)
# vec_y = vec_y / np.linalg.norm(vec_y)

# vec_x = np.cross(vec_y, vec_z)
# vec_x = vec_x / np.linalg.norm(vec_x)

# p1 = np.array([-Lx/2, 0, 0])
# p2 = np.array([+Lx/2, 0, 0])

# p1y1 = np.array([-Lx/2 - x_eps, +Ly, 0])
# p1y2 = np.array([-Lx/2 - x_eps, -Ly, 0])

# p2y1 = np.array([Lx/2 + x_eps, +Ly, 0])
# p2y2 = np.array([Lx/2 + x_eps, -Ly, 0])

# y1_mid = 0.5*(p1y1 + p2y1)
# y2_mid = 0.5*(p1y2 + p2y2)
# # ==========================


# rot = np.column_stack((vec_x, vec_y, vec_z))

# p1 = np.dot(rot, p1)
# p2 = np.dot(rot, p2)

# p1y1 = np.dot(rot, p1y1)
# p1y2 = np.dot(rot, p1y2)

# p2y1 = np.dot(rot, p2y1)
# p2y2 = np.dot(rot, p2y2)

# y1_mid = np.dot(rot, y1_mid)
# y2_mid = np.dot(rot, y2_mid)

# # translate to the origin   
# p1 = p1 + np.array([x0, y0, z0])
# p2 = p2 + np.array([x0, y0, z0])

# p1y1 = p1y1 + np.array([x0, y0, z0])
# p1y2 = p1y2 + np.array([x0, y0, z0])

# p2y1 = p2y1 + np.array([x0, y0, z0])
# p2y2 = p2y2 + np.array([x0, y0, z0])

# y1_mid = y1_mid + np.array([x0, y0, z0])
# y2_mid = y2_mid + np.array([x0, y0, z0])

# p1 = gmsh.model.occ.addPoint(*p1)
# p1y1 = gmsh.model.occ.addPoint(*p1y1)
# p1y2 = gmsh.model.occ.addPoint(*p1y2)
# y1_mid = gmsh.model.occ.addPoint(*y1_mid)
# gmsh.model.occ.synchronize()


# p2 = gmsh.model.occ.addPoint(*p2)
# p2y1 = gmsh.model.occ.addPoint(*p2y1)
# p2y2 = gmsh.model.occ.addPoint(*p2y2)
# y2_mid = gmsh.model.occ.addPoint(*y2_mid)
# gmsh.model.occ.synchronize()

# arc1 = gmsh.model.occ.addCircleArc(p1y1, p1, p1y2)
# # ly2  = gmsh.model.occ.addLine(p1y2,p2y2)
# ly2_1 = gmsh.model.occ.addLine(p1y2,y2_mid)
# ly2_2 = gmsh.model.occ.addLine(y2_mid,p2y2)

# arc2 = gmsh.model.occ.addCircleArc(p2y2, p2,p2y1)
# # ly1_mid  = gmsh.model.occ.addLine(p2y1,p1y1)
# ly1_1 = gmsh.model.occ.addLine(p2y1,y1_mid)
# ly1_2 = gmsh.model.occ.addLine(y1_mid,p1y1)


# # create the surface
# curve_loop = gmsh.model.occ.addCurveLoop([arc1, ly2_1,ly2_2, arc2, ly1_1, ly1_2])

# # cerrar curve 

# # wire1 = gmsh.model.occ.addWire([curve_loop],checkClosed=True)

# ss = gmsh.model.occ.addPlaneSurface([curve_loop])

# gmsh.model.occ.synchronize()

In [19]:
# gmsh.fltk.run()